In [12]:
%matplotlib inline

In [13]:
!hostname

mum-hpc2-gpu2


In [14]:
%cd ICRA_2023

[Errno 2] No such file or directory: 'ICRA_2023'
/home/ssap0002/ICRA_2023


In [15]:
from VRNN_DAT import VRNN_DAT
from RNN import GRU, LSTM

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt 

## Data Loading

In [16]:
import os
import numpy as np
import pickle5 as pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ORI_DATA_DIR = 'data/gripper/full'
HALF_DATA_DIR = 'data/gripper/half'
SYNTH_DATA_DIR = 'output/'

In [23]:
def to_timeseries(X, y, time_steps = 50):
    dataX = []
    dataY = []
    # Cut data by sequence length
    for i in range(0, len(X) - time_steps):
        _x = X[i:i+time_steps, :]
        _y = y[i:i+time_steps, :]
        dataX.append(_x)
        dataY.append(_y)
    return np.array(dataX), np.array(dataY)

def revert_timeseries(X, time_steps = 100):
  # if len(X[0]) == time_steps and time_steps != 1:
  #   return np.array(X[0])
  head = X[0]
  tail = np.array([f[time_steps-1] for f in X])
  return np.concatenate((head, tail))

def load_pickle(filepath):
    with open(filepath,'rb') as f:
      x = pickle.load(f)
      return x
    
def _preprocess(data):
  # Features
  flex = data['Flex'][0]
  pressure = data['Pressure'][0]

  # Labels
  f1, f2 = data['Force']
  x1, x2, x3, x4, x5, x6, x7, x8, x9, x10 = data['PosX']
  y1, y2, y3, y4, y5, y6, y7, y8, y9, y10 = data['PosY']
      
  x = list(zip(flex, pressure, f1, f2, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, y1, y2, y3, y4, y5, y6, y7, y8, y9, y10))
  return np.array(x)

def shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def load_data(s, _from='Train'):
  raw_data = load_pickle("data/new_soro_all.pkl")
  train = raw_data.xs(_from, axis=0, level=1, drop_level=True)

  raw_data = _preprocess(train[s])

  label_scaler = MinMaxScaler()
  feature_scaler = MinMaxScaler()
    
  features, labels = feature_scaler.fit_transform(raw_data[:, :2]), label_scaler.fit_transform(raw_data[:, 2:])   

  # Timeseries
  X_train, y_train = to_timeseries(features, labels, params['timesteps'])

  return  X_train, y_train, (label_scaler, feature_scaler)

In [29]:
def load_dataset(datatype, datasets, batch_size, no_state_data=[]):
    X_train, y_train, domain_labels = [], [], []
    
    if datatype == 'test':
        for scenario in datasets:
            _x, _y, scaler = load_data(scenario, _from='Test')
            X_train.append(np.array(_x).astype('float32')), y_train.append(np.array(_y).astype('float32'))
            if scenario in no_state_data:
                domain_labels.append([0 for _ in range(len(_x))])
            else:
                domain_labels.append([1 for _ in range(len(_x))])

        for i, (x,y) in enumerate(zip(X_train, y_train)):
            if i == 0:
                dataset = tf.data.Dataset.from_tensor_slices((x, y, domain_labels[i]))
                dataset = dataset.batch(batch_size)
            
            else: 
                _d = tf.data.Dataset.from_tensor_slices((x, y, domain_labels[i]))
                _d = _d.batch(batch_size)
                dataset = dataset.concatenate(_d)
        return dataset, scaler
    
    elif datatype == 'train':
        for scenario in datasets:
            _x, _y, scaler = load_data(scenario, _from='Train')
            X_train.append(np.array(_x).astype('float32')), y_train.append(np.array(_y).astype('float32'))
            if scenario in no_state_data:
                domain_labels.append([0 for _ in range(len(_x))])
            else:
                domain_labels.append([1 for _ in range(len(_x))])

        Xdata = np.concatenate(X_train)
        ydata = np.concatenate(y_train)
        Statedata = np.concatenate(domain_labels).astype(np.float32)

        # Shuffle
        p = np.random.permutation(len(Xdata))
        Xdata = Xdata[p]
        ydata = ydata[p]
        Statedata = Statedata[p]

        dataset =  tf.data.Dataset.from_tensor_slices((Xdata ,ydata, Statedata))
        return dataset.batch(batch_size), scaler

In [30]:
def get_dataset(model_type, train_scenario, batch_size):
    all_datasets = ['osc_tip_30', 'rand_tip_30', 'osc_rand_30', 'rand_rand_60']
    
    if train_scenario == "tip_contact":
        _train_d = ['osc_tip_30', 'rand_tip_30']
    elif train_scenario == "random_contact":
        _train_d = ['osc_rand_30', 'rand_rand_60']
    
    _test_d = [d for d in all_datasets if d not in _train_d]
    
    test_dataset, (label_scaler, feature_scaler) = load_dataset('test', _test_d, batch_size)
        
    if model_type == "GRU" or model_type == "LSTM":           
        train_dataset, _ = load_dataset('train', _train_d, batch_size)
    
    elif model_type == "VRNN_DAT":
        train_dataset, _ = load_dataset('train', all_datasets, batch_size, no_state_data=_test_d)
    
    return train_dataset, test_dataset, (label_scaler, feature_scaler)

def get_model(model_type, params=None):
    if model_type == "GRU":
        return GRU(params['x_dim'], params['timesteps'], params['z_dim'], params['h_dim'])
    elif model_type == "LSTM":
        return LSTM(params['x_dim'], params['timesteps'], params['z_dim'], params['h_dim'])
    elif model_type == "VRNN":
        pass
    elif model_type == "VRNN_DAT":
        return VRNN_DAT(params['x_dim'], params['h_dim'], params['z_dim'], phi_x_dim=params['phi_x_dim'], d_dim=params['d_dim'], vrnn_lr=params['lr'], adv_lr=params['adv_lr'], clip=params['clip'], rnn=params['rnn'])

def evaluate_model(model, test_dataset):
    pred, y_test = [], []
    for step, (x_batch_test, y_batch_test, _) in enumerate(test_dataset):
        _pred = model.predict(x_batch_test)
        pred.append(_pred)
        y_test.append(y_batch_test)
        
    pred = np.concatenate(pred)
    y_test = np.concatenate(y_test)
    
    force_x_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    force_z_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    marker_rmse_metric = tf.keras.metrics.RootMeanSquaredError()
    
    force_x_rmse_metric.update_state(y_test[:, :, 0], pred[:, :, 0]).numpy()
    force_z_rmse_metric.update_state(y_test[:, :, 1], pred[:, :, 1]).numpy()
    
    force_rmse = (force_x_rmse_metric.result().numpy(), force_z_rmse_metric.result().numpy())

    marker_rmse_metric.update_state(y_test[:, :, 2:], pred[:, :, 2:]).numpy()
    marker_rmse = marker_rmse_metric.result().numpy()
    
    return force_rmse, marker_rmse

## Hyperparameters

In [31]:
x_dim = 2 # flex + pressure
h_dim = 128
z_dim = 22 # 2 force + 10*2D coordinates
phi_x_dim = 32
d_dim = 128
n_epochs = 50
clip = 5
lr = 1e-3
adv_lr = 1e-3
batch_size = 128 # 32
seed = 261199
timesteps=100
rnn="LSTM"
# tf.random.set_seed(seed)

In [32]:
from sorcery import dict_of

params = dict_of(x_dim, h_dim, z_dim, phi_x_dim, timesteps, d_dim, n_epochs, clip, lr, adv_lr, batch_size, rnn)

## Train Models

In [33]:
model_types = ["VRNN_DAT"]
# model_types = ["LSTM"]

scenarios = ['tip_contact', 'random_contact']
df = pd.DataFrame(columns=["Model", "Train Scenario", "Force_X_RMSE", "Force_Z_RMSE", "Marker_RMSE", "Run"])
                  
for model_type in model_types:
    for train_scenario in scenarios:
        if train_scenario == "tip_contact":
            mn = 3
        elif train_scenario == "random_contact":
            mn = 1
        for run in range(mn, 6):
            print(f"================={model_type}_LSTM:{train_scenario}:{run}==================")
            # Prepare dataset and model
            train_dataset, test_dataset, scalers = get_dataset(model_type, train_scenario, params['batch_size'])
            model = get_model(model_type, params)

            # Train
            if model_type == "GRU" or model_type == "LSTM":
                model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics=[tf.metrics.RootMeanSquaredError()])
            else:
                model.compile()
            model.fit(train_dataset, epochs=params['n_epochs'], batch_size=params['batch_size'])

            # Save weights
            model.save_weights(f"weights/{model_type}_LSTM/{train_scenario}/run_{run}/")

            # Evaluate
            (force_x_rmse, force_z_rmse), marker_rmse = evaluate_model(model, test_dataset)

            # Write to df
            df = df.append({"Model": model_type, "Train Scenario": train_scenario, 'Force_X_RMSE': force_x_rmse, 'Force_Z_RMSE': force_z_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)
            
            print(f"================={model_type}_LSTM:{train_scenario}:{run}:{force_x_rmse}:{force_z_rmse}:{marker_rmse}==================")

=================VRNN_DAT_LSTM:tip_contact:3==================
Using LSTM Backend
Epoch 1/50


2022-09-05 22:45:48.138170: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] function_optimizer failed: INVALID_ARGUMENT: Input 0 of node StatefulPartitionedCall/zeros_like_2708 was passed float from StatefulPartitionedCall/sequential_24/gru_5/PartitionedCall:6 incompatible with expected variant.
2022-09-05 22:45:49.284699: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] shape_optimizer failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 25
2022-09-05 22:45:49.911606: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] layout failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 25
2022-09-05 22:45:50.333665: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] remapper failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 25
2022-09-05 22:45:52.101421: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] tfg_optimizer{} failed: INVALID_ARGUMENT: Input 0 of node StatefulPartitionedCall/zeros_like_2708 was pa

583/583 [==============================] - 402s 367ms/step - loss: -73982.8828 - rmse: 0.2592
Epoch 2/50
583/583 [==============================] - 214s 366ms/step - loss: -131097.1406 - rmse: 0.1629
Epoch 3/50
583/583 [==============================] - 213s 366ms/step - loss: -149375.9375 - rmse: 0.1519
Epoch 4/50
583/583 [==============================] - 214s 367ms/step - loss: -164634.5781 - rmse: 0.1430
Epoch 5/50
583/583 [==============================] - 214s 367ms/step - loss: -176203.3281 - rmse: 0.1360
Epoch 6/50
583/583 [==============================] - 213s 365ms/step - loss: -184679.2344 - rmse: 0.1316
Epoch 7/50
583/583 [==============================] - 213s 366ms/step - loss: -192352.6094 - rmse: 0.1281
Epoch 8/50
583/583 [==============================] - 213s 365ms/step - loss: -198229.4062 - rmse: 0.1250
Epoch 9/50
583/583 [==============================] - 214s 366ms/step - loss: -204154.6406 - rmse: 0.1222
Epoch 10/50
583/583 [==============================] - 213

/tmp/ipykernel_1872459/1992938768.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": train_scenario, 'Force_X_RMSE': force_x_rmse, 'Force_Z_RMSE': force_z_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


=================VRNN_DAT_LSTM:tip_contact:3:0.15542836487293243:0.1766979694366455:0.11768753826618195==================
=================VRNN_DAT_LSTM:tip_contact:4==================
Using LSTM Backend
Epoch 1/50


...

/tmp/ipykernel_1872459/1992938768.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": train_scenario, 'Force_X_RMSE': force_x_rmse, 'Force_Z_RMSE': force_z_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


=================VRNN_DAT_LSTM:random_contact:4:0.12716524302959442:0.16555415093898773:0.162543386220932==================
=================VRNN_DAT_LSTM:random_contact:5==================
Using LSTM Backend
Epoch 1/50


2022-09-06 19:55:03.183545: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] function_optimizer failed: INVALID_ARGUMENT: Input 0 of node StatefulPartitionedCall/zeros_like_2708 was passed float from StatefulPartitionedCall/sequential_59/gru_12/PartitionedCall:6 incompatible with expected variant.
2022-09-06 19:55:04.369506: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] shape_optimizer failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 25
2022-09-06 19:55:05.006474: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] layout failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 25
2022-09-06 19:55:05.434359: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] remapper failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 25
2022-09-06 19:55:07.241805: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] tfg_optimizer{} failed: INVALID_ARGUMENT: Input 0 of node StatefulPartitionedCall/zeros_like_2708 was p

583/583 [==============================] - 408s 368ms/step - loss: -111762.7656 - rmse: 0.2448
Epoch 2/50
583/583 [==============================] - 215s 368ms/step - loss: -190040.0000 - rmse: 0.1569
Epoch 3/50
583/583 [==============================] - 212s 364ms/step - loss: -219270.1406 - rmse: 0.1456
Epoch 4/50
583/583 [==============================] - 212s 364ms/step - loss: -238618.1875 - rmse: 0.1374
Epoch 5/50
583/583 [==============================] - 213s 365ms/step - loss: -253757.7500 - rmse: 0.1318
Epoch 6/50
583/583 [==============================] - 214s 366ms/step - loss: -266368.3125 - rmse: 0.1280
Epoch 7/50
583/583 [==============================] - 213s 365ms/step - loss: -276888.9375 - rmse: 0.1248
Epoch 8/50
583/583 [==============================] - 213s 365ms/step - loss: -285397.2812 - rmse: 0.1222
Epoch 9/50
583/583 [==============================] - 212s 363ms/step - loss: -292596.4688 - rmse: 0.1199
Epoch 10/50
583/583 [==============================] - 21

/tmp/ipykernel_1872459/1992938768.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Model": model_type, "Train Scenario": train_scenario, 'Force_X_RMSE': force_x_rmse, 'Force_Z_RMSE': force_z_rmse, 'Marker_RMSE': marker_rmse, "Run": run}, ignore_index=True)


In [34]:
df.head()

,Model,Train Scenario,Force_X_RMSE,Force_Z_RMSE,Marker_RMSE,Run
0,VRNN_DAT,tip_contact,0.155428,0.176698,0.117688,3
1,VRNN_DAT,tip_contact,0.154345,0.171680,0.118707,4
2,VRNN_DAT,tip_contact,0.177723,0.176812,0.122758,5
3,VRNN_DAT,random_contact,0.136882,0.165781,0.172490,1
4,VRNN_DAT,random_contact,0.130915,0.159382,0.168816,2


In [35]:
df.to_csv('vrnn_dat_lstm_results.csv', index=False)